In [ ]:
import numpy as np 
from PIL import Image
import torch
import matplotlib.pyplot as plt
import torchvision 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

img = Image.open("../input/cat-and-dog/test_set/test_set/cats/cat.4001.jpg")
img = np.array(img)
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self, path, size, transform):
        super().__init__()
        self.path = path
        self.size = 2 * size
        self.transform = transform
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        if index%2 ==0:
            p = self.path + "cats/cat."
            label = 1
        else:
            p = self.path + "dogs/dog."
            label = 0
        index = index//2 + 1
        p = p + str(index) + ".jpg"
        img = Image.open(p)
        img = self.transform(img)
        label = torch.tensor(label,dtype=torch.float)
        return img,label 
    
transform = transforms.Compose([
    transforms.Resize((150,150)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
])
path = "../input/cat-and-dog/training_set/training_set/"
dataset = myDataset(path=path, size=4000, transform=transform)
dataLoader = DataLoader(dataset,batch_size=32,num_workers=8,shuffle=True,drop_last=True,pin_memory=True)

path = "../input/cat-and-dog/test_set/test_set/"
testset = myDataset(path=path, size=1000, transform=transform)
testLoader = DataLoader(dataset,batch_size=32,num_workers=8,shuffle=True,drop_last=False,pin_memory=True)

print("finish loading")

In [ ]:
class FCNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Sequential(nn.Linear(3*150*150,3*20*20), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(3*20*20,400), nn.ReLU())
        self.fc3 = nn.Sequential(nn.Linear(400,20), nn.ReLU())
        self.fc4 = nn.Sequential(nn.Linear(20,1), nn.Sigmoid())
        
    def forward(self,x):
        #torch.Size([512, 3, 150, 150])
        x = x.view(-1,3*150*150)
        #torch.Size([512, 67500])
        x = self.fc1(x)
        #torch.Size([512, 1200])
        x = self.fc2(x)
        #torch.Size([512, 400])
        x = self.fc3(x)
        #torch.Size([512, 20])
        x = self.fc4(x)
        #torch.Size([512, 1])
        return x.view(-1)

class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3,32,3),    nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(32,64,3),   nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(64,128,3),  nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv4 = nn.Sequential(nn.Conv2d(128,128,3), nn.ReLU(), nn.MaxPool2d(2,2))
       
        self.fc1 = nn.Sequential(nn.Linear(6272,512), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(512,1), nn.Sigmoid())

    def forward(self,x): 
        batch = x.shape[0]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        x = x.view(batch,-1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        return x.view(-1)


In [ ]:
class Classifier():
    def __init__(self):
        super().__init__()
        self.learning_rate = 0.001
        self.epoch = 10
        self.model = ConvNet()
        self.criterion = self.criterion = torch.nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        if torch.cuda.is_available():
            self.model = self.model.cuda()
            self.criterion = self.criterion.cuda()
            
    def train(self):
        for e in range(self.epoch):
            for i,data in enumerate(dataLoader):
                x,y = data
                if torch.cuda.is_available():
                    x = x.cuda()
                    y = y.cuda()
                    
                pred = self.model(x)
                loss = self.criterion(pred,y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            if e % 1 == 0:
                print("epoch:{}, loss:{}".format(e, loss.data))
        
    def test(self):
        cnt = 0
        tot = 0
        for i,data in enumerate(testLoader):
            x,y = data
            if torch.cuda.is_available():
                x,y = x.cuda(), y.cuda()
            with torch.no_grad():
                pred = self.model(x)
            loss = self.criterion(pred,y)
            pred = torch.sign(pred-0.5)==1
            cnt += sum(pred==y) 
            tot += len(y)
        accuracy = cnt / tot
        print("accuracy: ", accuracy)


Model = Classifier()
Model.train()
Model.test()

准确率达到了97.99%